In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Two-level system

In [ ]:
# One-qubit gates
I = np.array([[1, 0], [0, 1]])
Z = np.array([[1, 0], [0, -1]])
X = np.array([[0, 1], [1, 0]])

def hamiltonian_general_tls(lam):
    # Basic params
    E1 = 0
    E2 = 4
    V11 = 3
    V22 = -V11
    Vx = 0.2
    #H_0 params
    H_0_I = (E1 + E2)/2
    H_0_Z= (E1 - E2)/2
    #H_I params
    H_I_I = (V11 + V22)/2
    H_I_Z = (V11-V22)/2
    H_I_X = Vx
    #H_0 and H_I
    H_0 = H_0_I*I + H_0_Z*Z
    H_I = H_I_I*I + H_I_Z*Z + H_I_X*X
    return H_0 + lam*H_I

def solve_tls_vs_lambda(lam_max=1):
    #Array of different interaction strengths
    lam = np.linspace(0, lam_max, 100)
    #Arrays to hold eigenvecs and eigenvals
    eig_vals = np.zeros((2, len(lam)))
    eig_vecs = np.zeros((2, 2, len(lam)), dtype = np.complex128)
    
    for i in range(len(lam)):
        H = hamiltonian_general_tls(lam[i])
        val, vec = np.linalg.eigh(H)
        eig_vals[:, i] = val
        eig_vecs[:, :, i] = vec
    return lam, eig_vals, eig_vecs

In [ ]:
### Simple demo
lam, vals, vecs = solve_tls_vs_lambda(4/3)

In [ ]:
plt.plot(lam, vals[0,:])
plt.plot(lam, vals[1,:])

### Two-qubit system

In [ ]:
# One-qubit gates
I = np.array([[1, 0], [0, 1]])
Z = np.array([[1, 0], [0, -1]])
X = np.array([[0, 1], [1, 0]])
# Two-qubit gates
II = np.kron(I, I)
ZI = np.kron(Z, I)
IZ = np.kron(I, Z)
ZZ = np.kron(Z, Z)
XX = np.kron(X, X)

def hamiltonian_two_qubits(lam):
    #Basic params
    E1 = 0.0
    E2 = 2.5
    E3 = 6.5
    E4 = 7.0
    Vx = 2.0
    Vz = 3.0
    #H_0 params
    E_II = (E1 + E2 + E3 + E4)/4
    E_ZI = (E1 + E2 - E3 - E4)/4
    E_IZ = (E1 - E2 + E3 - E4)/4
    E_ZZ = (E1 - E2 - E3 + E4)/4
    #H_0 and H_I
    H_0 = E_II*II + E_ZI*ZI + E_IZ*IZ + E_ZZ*ZZ
    H_I = Vx*XX + Vz*ZZ
    return H_0 + lam*H_I

def solve_two_qubit_vs_lambda(lam_max=1):
    #Array of different interaction strengths
    lam = np.linspace(0, lam_max, 100)
    #Arrays to hold eigenvecs and eigenvals
    eig_vals = np.zeros((4, len(lam)))
    eig_vecs = np.zeros((4, 4, len(lam)), dtype = np.complex128)
    
    for i in range(len(lam)):
        H = hamiltonian_two_qubits(lam[i])
        val, vec = np.linalg.eigh(H)
        eig_vals[:, i] = val
        eig_vecs[:, :, i] = vec
    return lam, eig_vals, eig_vecs

In [ ]:
#Simple demo
lam, vals, vecs = solve_two_qubit_vs_lambda(4/3)

In [ ]:
plt.plot(lam, vals[0,:])
plt.plot(lam, vals[1,:])
plt.plot(lam, vals[2,:])
plt.plot(lam, vals[3,:])

### Von Neumann entropy

In [ ]:
# One-qubit basis, needed for partial trace
def qubit_0():
    return np.array([1, 0], dtype=np.complex128)
def qubit_1():
    return np.array([0, 1], dtype=np.complex128)

def rho_vs_lambda(state_vs_lambda):
    #Initialize matrix to hold density matrix for each lambda.
    #Density matrix is 4x4 for the two-qubit system
    rho = np.zeros((4, 4, len(state_vs_lambda[0, :])), dtype = np.complex128)
    for i in range(len(state_vs_lambda[0, :])):
        #Density matrix is outer product of state |psi>, i.e. rho = |psi><psi|.
        #Second factor is bra-vector, i.e. conjugated.
        rho[:, :, i] = np.outer(state_vs_lambda[:, i], state_vs_lambda[:, i].conj())
    return rho

def get_rho_A(rho):
    #Calculate reduced density matrix, like rho_A = tr_B(rho).
    b0 = np.kron(I, qubit_0())
    b1 = np.kron(I, qubit_1())
    return b0.conj()@rho@b0.T + b1.conj()@rho@b1.T

def entropy_vs_lambda(rho_vs_lambda):
    #Initialize 1D vector to hold entropy value at different lambda
    entropy = np.zeros(len(rho_vs_lambda[0,0, :]), dtype = np.complex128)
    for i in range(len(rho_vs_lambda[0,0, :])):
        #Get the reduced density matrix of subsystem A
        rho_A = get_rho_A(rho_vs_lambda[:, :, i])
        #Get eigenvalues of rho_A
        vals = np.linalg.eigvalsh(rho_A)
        #Check that value is not 0 before adding to entropy, since log(0) is not defined.
        for val in vals:
            if val > 0.0001:
                entropy[i] -= val*np.log2(val)
    return entropy

In [ ]:
# Demo
lam, vals, vecs = solve_two_qubit_vs_lambda(4/3)
rho = rho_vs_lambda(vecs[:,0,:])
entropy = entropy_vs_lambda(rho)

In [ ]:
plt.plot(lam, entropy)

### Lipkin Model

In [ ]:
def H_e(V):
    return np.array([[-2, np.sqrt(6)*V, 0, 0], [np.sqrt(6)*V, 0, 0, np.sqrt(6)*V], [0, 0, 0, 0], [0,np.sqrt(6)*V, 0, 2]])

def H_o(V):
    return np.array([[-1, 3*V], [3*V, 1]])

def solve_lipkin_energies_vs_lambda(lam_max=1):
    #Array of different interaction strengths
    lam = np.linspace(0, lam_max, 100)
    #Arrays to hold eigenvals for H_e and H_o
    eig_vals_e = np.zeros((4, len(lam)))
    eig_vals_o = np.zeros((2, len(lam)))
    
    for i in range(len(lam)):
        val_e = np.linalg.eigvalsh(H_e(lam[i]))
        eig_vals_e[:, i] = val_e
        val_o = np.linalg.eigvalsh(H_o(lam[i]))
        eig_vals_o[:, i] = val_o
    return lam, eig_vals_e, eig_vals_o

In [ ]:
#Demo
lam, vals_e, vals_o = solve_lipkin_energies_vs_lambda(4/3)

In [ ]:
plt.plot(lam, vals_e[0,:])
plt.plot(lam, vals_e[1,:])
plt.plot(lam, vals_e[3,:])
plt.plot(lam, vals_o[0,:])
plt.plot(lam, vals_o[1,:])